In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [5]:
class MyCLRuleMonitor(tf.keras.callbacks.Callback):
  def __init__(self,CL):
    super(MyCLRuleMonitor,self).__init__()
    self.CL = CL


  def on_epoch_end(self,epoch,logs=None):
    testScore = logs['val_r2_score']
    trainScore = logs['r2_score']

    if testScore > trainScore and testScore >= self.CL:
      self.model.stop_training = True

In [6]:
data = pd.read_csv('50_Startups.csv')
# print(data.head())
print(data.info())
data.dropna(inplace=True)
print(data.info())

# 1. Features and Labels split
features = data.iloc[:, [0,1,2,3]].values
labels = data.iloc[:, [4]].values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB
None


In [7]:
# Transform the State values using One Hot Encoder concat it with the featues and then process

ohe = OneHotEncoder(sparse_output=False)
ohe.fit(features[:,[3]])
states = ohe.fit_transform(features[:,[3]])
print(states)
final_featues = np.concatenate((states, features[:,[0,1,2]]),axis=1)
print(final_featues)

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
[[0.0 0.0 1.0 165349.2 136897.8 471784.1]
 [1.0 0.0 0.0 162597.7 151377.59 443898.53]
 [0.0 1.0 0.0 153441.51 101145.55 407934.54]
 [0.0 0.0 1.0 144372.41 118671.85 383199.62]
 [0.0 1.0 0.0 142107.34 91391.77 366168.42]
 [0.0 0.0 1.0 131876.9 99814.71 362861.36]
 [1.0 0.0 0.0 134615.46 147198.87 127716.82]
 [0.0 1.0 0.0 130298.13 145530.06 323876.68]
 [0.0 0.0 1.0 120542.52 148718.95 311613.29]
 

In [8]:
rs = RobustScaler()
s_features = rs.fit_transform(final_featues)

# b. Labels need to be scaled using Min Max Scaler
mn = MinMaxScaler()
m_labels = mn.fit_transform(labels)

In [9]:
# 3. Train test split
X_train, X_test, y_train, y_test = train_test_split(s_features, m_labels, test_size=0.2, random_state = 10)

In [11]:
# Deep learning model steps:
# 1. Model architeching (Creating Input/Hidden and Output layers)
# 2. Model compile
# 3. Model fit (Training)
# 4. Model evaluation
# 5. Model deployment

# https://www.tensorflow.org/api_docs/python/tf/keras/Model
# Model architecting:


model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(units = 100, activation='relu', input_shape=(6,)))
# Handle the no of feature cols and activation function as per rules.
model.add(tf.keras.layers.Dense(units= 100, activation='relu'))
model.add(tf.keras.layers.Dense(units= 100, activation='relu'))
model.add(tf.keras.layers.Dense(units= 1, activation='linear'))

# 2. Model compile:
model.compile(optimizer = 'adam',loss = 'mean_squared_error' ,metrics = [tf.keras.metrics.R2Score()])


# 3. Fit the model/ Train
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs= 100000 ,callbacks=[MyCLRuleMonitor(.85)])

Epoch 1/100000
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 632ms/step - loss: 0.2784 - r2_score: -4.7224 - val_loss: 0.1569 - val_r2_score: -1.7275
Epoch 2/100000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - loss: 0.1431 - r2_score: -1.9419 - val_loss: 0.0806 - val_r2_score: -0.4009
Epoch 3/100000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - loss: 0.0761 - r2_score: -0.5643 - val_loss: 0.0503 - val_r2_score: 0.1252
Epoch 4/100000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - loss: 0.0485 - r2_score: 0.0035 - val_loss: 0.0415 - val_r2_score: 0.2785
Epoch 5/100000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - loss: 0.0445 - r2_score: 0.0846 - val_loss: 0.0335 - val_r2_score: 0.4174
Epoch 6/100000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step - loss: 0.0392 - r2_score: 0.1941 - val_loss: 0.0200 - val_r2_score: 0.6524
Epoch 7/100000
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 243ms/step - loss: 0.0285 - r2_score: 0.4144 - val_loss: 0.0107 - val_r2_score: 0.8137
Epoch 8/100000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step - loss: 0.0172 - r2_score: 0.64

In [12]:

# Testing app: Reading inputs
rdspends = float(input('Enter the r&d spends: '))
adminspends = float(input('Enter the admin spends: '))
marketingspends = float(input('Enter the marketing spends: '))
state = input('Enter the location: ')

# Encode the state value:
state_values = ohe.categories_[0]
while state not in state_values:
   state = input('Enter the location: ')

# Transform this value:
input_state = ohe.transform(np.array([[state]]))
input_final_features = np.concatenate((input_state, np.array([[rdspends, adminspends, marketingspends]])), axis = 1)

# Scale the features:
scaled_input = rs.fit_transform(input_final_features)
predicted_values = model.predict(scaled_input)

# reverse transform the predicted value:
profit  = mn.inverse_transform(predicted_values)

print(f'For values: {rdspends},{adminspends}, {marketingspends}, and {state}: Expected profit is: {profit}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
For values: 1000.0,2322.0, 4433.0, and New York: Expected profit is: [[38238.273]]
